# Cuckoo filter
_Reference: Fan, Andersen, Kaminsky & Mitzenmacher: "Cuckoo Filter: Practically Better Than Bloom"_

__Note: See also the section: "Cuckoo versus Bloom filters"__

## What it is
A cuckoo filter is conceptually similar to a bloom filter, even though the underlying algorithm is quite different.

The cuckoo filter is similar to a `Set`. Hashable objects can be pushed into the filter, but objects cannot be extracted from the filter. Querying the filter, i.e. asking whether an object is in a filter is fast, but cuckoo filters has a certain probability of falsely returning `true` when querying about an object not actually in the filter. When querying an object that is in the filter, it is guaranteed to return `true`.

A cuckoo filter is defined by two parameters, `F` and its length `L`. Memory usage is proportional to `F`\*`L`, and the false positive rate is approximately proportional to N/(2^`F`\*`L`) where N is the number of elements in the filter.

### Pushing

Only hashable objects can be inserted into the filter. Inserting time is stochastic, but its expected duration is proportional to 1/(1 - N/`L`). To avoid infinite insertion times as N approaches `L`, an insert operation may fail if the filter is too full.

### Querying

Querying time is constant for all values of N, `F` and `L`. When querying about an object in the filter, it is guaranteed to return `true`. Querying about objects not in the filter returns `true` with a probability proportional to N/(2^`F`\*`L`).

In general, two distinct objects A and B will have a 1/(2^`F`-1) chance of sharing so-called "fingerprints". Independently, each object is assigned two "buckets" in the range 1:L/4, and inserted in an arbitrary of the two buckets. If objects A and B share fingerprints, and object A is in one of B's buckets, the existence of A will make a query for B return `true`, even when it's not in the filter.

### Deletion

Objects can be deleted from the filter. However, if object A and B collides in the filter, i.e. if querying A returns `true` when B is in the filter and vice versa, deleting A has a *might* also delete B. This happens if A and B share fingerprints and B is placed in one of A's buckets (see section: __Querying__.)

Deletion of objects is fast and constant time, except if the filter is at full capacity. In that case, it will attempt to self-organize after a deletion to allow new objects to be pushed. This takes some time.

### `FastCuckoo` and `SmallCuckoo`

Probably.jl comes with two different implementations of the cuckoo filter: `FastCuckoo` and `SmallCuckoo`. The latter uses a more complicated encoding scheme to achieve a slightly smaller memory footprint, but which also make all operations slower. The following plot shows how the speed of pushing objects depend on the load factor, i.e. how full the filter is, and how `FastCuckoo`s are ~2.5x faster than `SmallCuckoo`s, but that the `SmallCuckoo` uses about 10% less memory.  

![](cuckooperformance.png)

## Interface
